In [ ]:
import torch
from torch import nn
# define customized functions with customized gradients
class STEFunction_relay(torch.autograd.Function):
    """ define straight through estimator with overrided gradient (gate) """
    @staticmethod
    def forward(ctx, input_x, mask, threshold = 0.005):
        ctx.save_for_backward(input_x)
        mask_higher = (input_x > threshold).float()
        mask_lower = (input_x < (-1) * threshold).float()
        mask > 0
        return (input_x > 0).float()

    @staticmethod
    def backward(ctx, grad_output):
        input_x, = ctx.saved_tensors
        return torch.mul(F.softplus(input_x), grad_output)

In [7]:
now_state = 1
in_val = -0.015
threshold = 0.01
def Hysteresis(now_state, in_val, threshold):
    if now_state == 1:
        if in_val < (-1) * threshold:
            now_state = 0
    else:
        if in_val > threshold:
            now_state = 1
    return now_state
now_state = Hysteresis(now_state, in_val, threshold) 
print(now_state)

0


In [40]:
now_state = 1.0
in_val_list = [-0.011, 0.005, 0.001, 0.011, -0.009, -0.011, 0.0, 0.011]
threshold = 0.01
def Hysteresis(now_state, in_val, threshold):
    now_state = now_state * float(in_val > (-1) * threshold) + (1 - now_state) * float(in_val > threshold)
    return now_state
for in_val in in_val_list:
    now_state = Hysteresis(now_state, in_val, threshold) 
    print(now_state)

0.0
0.0
0.0
1.0
1.0
0.0
0.0
1.0


In [22]:
now_state = 0
in_val = -0.0101
threshold = 0.01
now_state = now_state * float(in_val > (-1) * threshold) + (1 - now_state) * float(in_val > threshold)
print(now_state)

0.0


In [43]:
import torch
from torch.nn import functional as F
class MultiInputRelayFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input1, input2):
        ctx.save_for_backward(input1, input2)
        return input1, input2

    @staticmethod
    def backward(ctx, grad_output1, grad_output2):
        input1, input2 = ctx.saved_tensors
        if not input1.requires_grad:
            grad_input1 = None
        else:
            grad_input1 = grad_output1.clone()

        if not input2.requires_grad:
            grad_input2 = None
        else:
            grad_input2 = grad_output2.clone()

        return grad_input1, grad_input2

x1 = torch.randn(10, requires_grad=True)
x2 = torch.randn(10)
print(x1)
# Use the MultiInputRelayFunction to calculate the output
output = MultiInputRelayFunction.apply(x1, x2)

# Calculate the loss
loss = (output[0] ** 2).sum()

# Backprop to calculate gradients
loss.backward()

# Update x1 using an optimizer
optimizer = torch.optim.Adam([x1], lr=0.1)
optimizer.step()
print(x1)

tensor([ 0.3044, -1.3046,  0.2962,  1.8289, -1.2884,  0.0434,  0.2522, -1.1956,
        -0.3554, -0.8371], requires_grad=True)
tensor([ 0.2044, -1.2046,  0.1962,  1.7289, -1.1884, -0.0566,  0.1522, -1.0956,
        -0.2554, -0.7371], requires_grad=True)


In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F
# define customized functions with customized gradients
class STEFunction_relay(torch.autograd.Function):
    """ define straight through estimator with overrided gradient (gate) """
    @staticmethod
    def forward(ctx, input, mask_old):
        ctx.save_for_backward(input)
        return mask_old

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        return torch.mul(F.softplus(input), grad_output), None

x1 = torch.randn(10, requires_grad=True)
nn.init.uniform_(x1, a = 0, b = 1)
mask_old = (x1 > 0).float()
print(x1)
print(mask_old)
mask_old.requires_grad = False
x1_out = STEFunction_relay.apply(x1, mask_old)
loss = torch.sum(x1_out)
threshold = 0.5

loss.backward()
optimizer = torch.optim.Adam([x1], lr=1)
optimizer.step()
print(x1)
print(mask_old)
mask_new = mask_old * (x1 > (-1) * threshold).float() + (1 - mask_old) * (x1 > threshold).float()
print(mask_new)

tensor([0.7832, 0.8368, 0.3353, 0.2943, 0.4404, 0.9394, 0.7239, 0.0102, 0.3909,
        0.6217], requires_grad=True)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([-0.2168, -0.1632, -0.6647, -0.7057, -0.5596, -0.0606, -0.2761, -0.9898,
        -0.6091, -0.3783], requires_grad=True)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([1., 1., 0., 0., 0., 1., 1., 0., 0., 1.])


In [20]:
import torch

# Create a tensor with random values
tensor = torch.rand(3, 3)

# Define a model with a single linear layer
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear = torch.nn.Linear(3, 3)

# Initialize the model
model = Model()

# Copy the values from the tensor into the model's parameters
print(model.linear.weight)
print(tensor)
model.linear.weight.data.copy_(tensor)
print(model.linear.weight)

### Other method to copy parameter from one to another
param1 = torch.nn.Parameter(torch.rand(3, 3))
param2 = torch.nn.Parameter(torch.rand(3, 3))
param2.data.copy_(param1.data)
print(param2.data)


Parameter containing:
tensor([[ 0.0808,  0.2326, -0.0325],
        [ 0.4869, -0.0527, -0.3543],
        [ 0.2577, -0.1948, -0.0866]], requires_grad=True)
tensor([[0.8084, 0.4958, 0.9062],
        [0.9594, 0.9866, 0.9635],
        [0.8957, 0.1373, 0.4079]])
Parameter containing:
tensor([[0.8084, 0.4958, 0.9062],
        [0.9594, 0.9866, 0.9635],
        [0.8957, 0.1373, 0.4079]], requires_grad=True)
tensor([[0.8566, 0.1845, 0.7607],
        [0.7572, 0.2837, 0.1152],
        [0.5088, 0.6485, 0.9269]])


In [22]:
class MyClass:
    def __init__(self):
        self._attr1 = Attr1()
    def set_attr1(self, value):
        self._attr1.attr2 = value

class Attr1:
    def __init__(self):
        self.attr2 = None

obj = MyClass()
obj.set_attr1("Value")
print(obj._attr1.attr2)
# setattr(eval('obj._attr1'), 'attr2', 'a')
print(getattr(obj, '_attr1').attr2)
getattr(obj, '_attr1').attr2 = 'a'
print(obj._attr1.attr2)
obj._attr1.attr2 = 'a'
print(obj._attr1.attr2)

Value
Value
a
a


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define your model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.w1 = nn.Parameter(torch.randn(1))
        self.w2 = nn.Parameter(torch.randn(1))
        self.b = nn.Parameter(torch.randn(1))

    def forward(self, x):
        fx = nn.functional.relu(x)
        # gx = self.w2 * (x ** 2) + self.w1 * x + self.b
        gx = self.w1 * x + self.b
        return fx, gx

# Define your loss function
def loss_fn(fx, gx):
    return (fx - gx).pow(2).mean()

net = Net()
# Define your optimizer
optimizer = optim.Adam([net.w1, net.w2, net.b], lr=0.001)
epochs = 10000
lr_scheduler_w = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer = optimizer, T_max = epochs)
# Train the model
for i in range(10000):
    # Generate a new random x with normal distribution
    x = torch.randn(32, 32) #N(0, 1)
    # x += torch.randn(32, 32) #N(0, 2)
    fx, gx = net(x)
    loss = loss_fn(fx, gx)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    lr_scheduler_w.step()
    print(loss.item())

2.5655179023742676
2.585801601409912
2.6732611656188965
2.5147557258605957
2.456362724304199
2.5252904891967773
2.6748785972595215
2.590719223022461
2.5767288208007812
2.695460796356201
2.381436586380005
2.4775819778442383
2.4465184211730957
2.658034324645996
2.6272432804107666
2.6138713359832764
2.516197919845581
2.4690756797790527
2.5446524620056152
2.4454331398010254
2.4559032917022705
2.528665065765381
2.571866989135742
2.521125078201294
2.588325262069702
2.3640542030334473
2.535177230834961
2.5767674446105957
2.4895098209381104
2.454501152038574
2.3956425189971924
2.344780683517456
2.463817834854126
2.5688986778259277
2.594576835632324
2.263411283493042
2.439621686935425
2.4333529472351074
2.395691394805908
2.383089065551758
2.3763115406036377
2.274500608444214
2.373952627182007
2.4002938270568848
2.4633586406707764
2.3370895385742188
2.4350249767303467
2.308439254760742
2.467555284500122
2.377220392227173
2.305999994277954
2.385093927383423
2.427910327911377
2.3043551445007324
2.

In [6]:
print(net.w2, net.w1, net.b)

Parameter containing:
tensor([-0.4746], requires_grad=True) Parameter containing:
tensor([0.5001], requires_grad=True) Parameter containing:
tensor([0.3989], requires_grad=True)


In [18]:
# x = torch.randn(20)
x = torch.tensor([3.0])
# x += torch.randn(20)
fx, gx = net(x)
print(x)
print(fx)
print(gx)

tensor([3.])
tensor([3.])
tensor([3.4935], grad_fn=<AddBackward0>)


In [26]:
x = torch.tensor([3.0])
x.var_map = 1
print(x.var_map)
fx, gx = net(x)
print(fx.var_map)

1


AttributeError: 'Tensor' object has no attribute 'var_map'

In [15]:
import torch
from torch.nn import functional as F
x = torch.randn(3, 3)
print(x)
mask = (torch.randn(3, 3) > 0).float()
print(mask)
out1 = F.relu(x * mask)
out2 =  mask* F.relu(x)
print(out1)
print(out2)
print(out1 - out2)


tensor([[ 1.3206,  0.8124,  0.7564],
        [ 0.5147,  0.3210, -0.5729],
        [ 1.0014,  0.7350, -0.8284]])
tensor([[0., 0., 0.],
        [0., 1., 0.],
        [0., 1., 1.]])
tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 0.3210, -0.0000],
        [0.0000, 0.7350, 0.0000]])
tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 0.3210, 0.0000],
        [0.0000, 0.7350, 0.0000]])
tensor([[0., 0., 0.],
        [0., 0., -0.],
        [0., 0., 0.]])


In [31]:
import torch
from torch.nn import functional as F
import torch.optim as optim

with torch.no_grad():
    x0 = torch.randn(4, 4)
    x1 = torch.clone(x0)
    mask0 = (torch.randn(4, 4) > 0).float()
    mask1 = torch.clone(mask0)
x0.requires_grad = True
x1.requires_grad = True
mask0.requires_grad = True
mask1.requires_grad = True
optimizer0 = optim.Adam([x0, mask0], lr=1)
optimizer1 = optim.Adam([x1, mask1], lr=1)
for i in range(1):
    out0 = F.relu(x0 * mask0)
    out1 =  mask1* F.relu(x1)
    loss0 = torch.sum(out0)
    loss1 = torch.sum(out1)
    print("difference between out0 and out1:", torch.sum(out0 - out1))
    optimizer0.zero_grad()
    optimizer1.zero_grad()
    loss0.backward()
    loss1.backward()
    print(mask0.grad)
    print(mask1.grad)
    optimizer0.step()
    optimizer1.step()
    # print(x0)
    # print(x1)
    # print(mask0)
    # print(mask1)

difference between out0 and out1: tensor(0., grad_fn=<SumBackward0>)
tensor([[-0.0000, 1.5212, 0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000, 0.0000],
        [-0.0000, -0.0000, 1.2987, -0.0000],
        [-0.0000, -0.0000, -0.0000, -0.0000]])
tensor([[0.0000, 1.5212, 1.0282, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.4364],
        [0.0000, 0.0000, 1.2987, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]])


In [33]:
import torch
x = torch.tensor([1, 2, 3])
x.size()
x.expand(4, 3)

tensor([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]])

In [1]:
import torch
import torch.nn as nn
size = 3
channel_size = [size]
para = nn.Parameter(torch.Tensor(*channel_size))
nn.init.uniform_(para, a = 0, b = 1)
print(para)
x_size = [2, 3, 2]
x_val = torch.Tensor(*x_size)
nn.init.uniform_(x_val, a = 0, b = 1)
print(x_val.shape)
print(x_val)
print(para.shape)
para_expand = para.view(1, para.shape[0], 1)
print("unsqueezed para shape:", para_expand.shape)
# para.expand_as(x_val)
# print("expanded para:", para)
print(para_expand)
print(para_expand*x_val)

Parameter containing:
tensor([0.2992, 0.9776, 0.0466], requires_grad=True)
torch.Size([2, 3, 2])
tensor([[[0.9096, 0.3820],
         [0.7862, 0.4860],
         [0.2381, 0.6455]],

        [[0.8634, 0.2339],
         [0.0743, 0.0943],
         [0.3034, 0.1583]]])
torch.Size([3])
unsqueezed para shape: torch.Size([1, 3, 1])
tensor([[[0.2992],
         [0.9776],
         [0.0466]]], grad_fn=<ViewBackward0>)
tensor([[[0.2721, 0.1143],
         [0.7686, 0.4751],
         [0.0111, 0.0301]],

        [[0.2583, 0.0700],
         [0.0726, 0.0922],
         [0.0141, 0.0074]]], grad_fn=<MulBackward0>)


/data3/hop20001/.conda/envs/torchenv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
para.data[0] = 1.2
print(para_expand)

tensor([[[1.2000],
         [0.9776],
         [0.0466]]], grad_fn=<ViewBackward0>)


In [3]:
para_cuda = para.cuda()

In [4]:
print(para_cuda.device)
print(para_expand.device)

cuda:0
cpu


In [5]:
para_expand_cuda = para_cuda.view(1, para.shape[0], 1)
print(para_expand_cuda.device)

cuda:0


In [8]:
para.data[0] = 1.3
para_cuda.data[0] = 1.4
print(para_expand)
print(para_expand_cuda)

tensor([[[1.3000],
         [0.9776],
         [0.0466]]], grad_fn=<ViewBackward0>)
tensor([[[1.4000],
         [0.9776],
         [0.0466]]], device='cuda:0', grad_fn=<ViewBackward0>)


In [10]:
para_cuda.copy_(para)
print(para_cuda)

tensor([1.3000, 0.9776, 0.0466], device='cuda:0', grad_fn=<CopyBackwards>)


In [14]:
import numpy as np
para_data_np = np.array([0., 1., 2.])
para_data_tensor = torch.from_numpy(para_data_np)
para_cuda.copy_(para_data_tensor)

tensor([0., 1., 2.], device='cuda:0', grad_fn=<CopyBackwards>)